In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import matplotlib.patches as mpatches

In [ ]:
# Dicionário de cores para cada tipo de ingrediente
tipo_cores = {
    'Carboidrato': 'green',
    'Proteína': 'blue',
    'Vegetal': 'orange',
    'Laticínio': 'purple',
    'Leguminosa': 'brown',
    'Gordura': 'pink',
    'Fruta': 'yellow',
    'Condimento': 'red'
}

In [ ]:
# Carregar o arquivo CSV
file_path = 'receitas.csv'  # Substitua pelo caminho correto do arquivo
df = pd.read_csv(file_path, sep=";")

In [ ]:
# Criar um grafo não direcionado
G = nx.Graph()
# Processar as receitas para adicionar os nós e arestas
for index, row in df.iterrows():
    ingredientes = row['ingredientes'].split(', ')
    tipos = row['tipos de ingredientes'].split(', ')
    
    # Adicionando nós com atributos (tipo de ingrediente)
    for i, ingrediente in enumerate(ingredientes):
        if not G.has_node(ingrediente):
            G.add_node(ingrediente, tipo=tipos[i])
    
    # Criar as arestas entre os ingredientes da mesma receita
    for i in range(len(ingredientes)):
        for j in range(i + 1, len(ingredientes)):
            G.add_edge(ingredientes[i], ingredientes[j])

In [ ]:
# Calcular o coeficiente de assortatividade por tipo
assortatividade_por_tipo = nx.attribute_assortativity_coefficient(G, 'tipo')
print(f'Coeficiente de assortatividade por tipo: {assortatividade_por_tipo}')

In [ ]:
# Exibir o total de nós e arestas
total_nos = len(G.nodes())
total_arestas = len(G.edges())
densidade = nx.density(G)
print(f"\nTotal de nós: {total_nos}")
print(f"Total de arestas: {total_arestas}")
print(f"Densidade da rede: {densidade:.4f}")

In [ ]:
# Diâmetro da rede (se a rede for conexa)
if nx.is_connected(G):
    diametro = nx.diameter(G)
    print(f"Diâmetro da rede: {diametro}")
else:
    print("A rede não é conexa, logo o diâmetro não pode ser calculado.")

In [ ]:
# Média do comprimento dos caminhos mais curtos
comprimentos_caminho = nx.all_pairs_shortest_path_length(G)
todos_caminhos = []
for source, paths in comprimentos_caminho:
    for target, length in paths.items():
        if source != target:
            todos_caminhos.append(length)
media_caminhos = np.mean(todos_caminhos)
print(f"Média do comprimento dos caminhos mais curtos: {media_caminhos:.2f}")

In [ ]:
# Visualizar a rede com layout de força
plt.figure(figsize=(14, 14))
pos = nx.spring_layout(G, seed=42, k=1.0)
node_colors = [tipo_cores.get(G.nodes[node]['tipo'], 'gray') for node in G.nodes()]
node_sizes = [G.degree(node) * 100 for node in G.nodes()]
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, edgecolors='black', alpha=0.7, linewidths=1.5)
nx.draw_networkx_edges(G, pos, alpha=0.3, width=0.5)
nx.draw_networkx_labels(G, pos, font_size=6, font_color='black')
plt.legend(handles=[mpatches.Patch(color=color, label=tipo) for tipo, color in tipo_cores.items()], loc='upper left', fontsize=10)
plt.title("Ingredient Network by Type")
plt.axis('off')
plt.show()